In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [105]:
dataset = pd.read_csv("heart.csv")
dataset.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [106]:
dataset.dtypes


age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [107]:
X = dataset.iloc[:, :-1]
X.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [108]:
y = dataset.iloc[:, [-1]]
y.head()


,target
0,1
1,1
2,1
3,1
4,1


In [109]:
X_ohe = pd.get_dummies(X, columns=["cp", "restecg", "slope", "thal"])


In [110]:
X_ohe.columns


Index(['age', 'sex', 'trestbps', 'chol', 'fbs', 'thalach', 'exang', 'oldpeak',
       'ca', 'cp_0', 'cp_1', 'cp_2', 'cp_3', 'restecg_0', 'restecg_1',
       'restecg_2', 'slope_0', 'slope_1', 'slope_2', 'thal_0', 'thal_1',
       'thal_2', 'thal_3'],
      dtype='object')

In [111]:
X_ohe = X_ohe.drop(["cp_0", "restecg_0", "slope_0", "thal_0"], axis=1)


In [112]:
X_ohe.head()


,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,ca,cp_1,cp_2,cp_3,restecg_1,restecg_2,slope_1,slope_2,thal_1,thal_2,thal_3
0,63,1,145,233,1,150,0,2.3,0,0,0,1,0,0,0,0,1,0,0
1,37,1,130,250,0,187,0,3.5,0,0,1,0,1,0,0,0,0,1,0
2,41,0,130,204,0,172,0,1.4,0,1,0,0,0,0,0,1,0,1,0
3,56,1,120,236,0,178,0,0.8,0,1,0,0,1,0,0,1,0,1,0
4,57,0,120,354,0,163,1,0.6,0,0,0,0,1,0,0,1,0,1,0


In [113]:
X_ohe.shape


(303, 19)

In [114]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)


In [115]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


In [116]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout


In [117]:
dnn = Sequential()
dnn.add(keras.Input(shape=(13,)))
dnn.add(Dense(512, activation="relu"))
dnn.add(Dropout(0.1))
dnn.add(Dense(256, activation="relu"))
dnn.add(Dropout(0.1))
dnn.add(Dense(256, activation="relu"))
# dnn.add(Dropout(0.1))
dnn.add(Dense(1, activation="sigmoid"))
dnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [118]:
dnn.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    batch_size=20,
    epochs=30,
)


Epoch 1/30
13/13 [==============================] - 1s 36ms/step - loss: 0.5255 - accuracy: 0.7521 - val_loss: 0.3456 - val_accuracy: 0.8525
Epoch 2/30
13/13 [==============================] - 0s 22ms/step - loss: 0.3698 - accuracy: 0.8264 - val_loss: 0.3575 - val_accuracy: 0.8525
Epoch 3/30
13/13 [==============================] - 0s 11ms/step - loss: 0.3277 - accuracy: 0.8760 - val_loss: 0.3459 - val_accuracy: 0.8852
Epoch 4/30
13/13 [==============================] - 0s 10ms/step - loss: 0.2915 - accuracy: 0.8884 - val_loss: 0.3597 - val_accuracy: 0.8525
Epoch 5/30
13/13 [==============================] - 0s 10ms/step - loss: 0.2666 - accuracy: 0.8719 - val_loss: 0.3509 - val_accuracy: 0.8852
Epoch 6/30
13/13 [==============================] - 0s 9ms/step - loss: 0.2421 - accuracy: 0.8967 - val_loss: 0.3772 - val_accuracy: 0.8689
Epoch 7/30
13/13 [==============================] - 0s 9ms/step - loss: 0.2239 - accuracy: 0.9132 - val_loss: 0.3472 - val_accuracy: 0.9016
Epoch 8/30
13/1

In [119]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


In [120]:
y_pred = dnn.predict(X_test)
dnn_predicted = y_pred > 0.5
dnn_conf_matrix = confusion_matrix(y_test, dnn_predicted)
dnn_acc_score = accuracy_score(y_test, dnn_predicted)
print("confussion matrix")
print(dnn_conf_matrix)
print("\n")
print("Accuracy of DNN:", dnn_acc_score * 100, "\n")
print(classification_report(y_test, dnn_predicted))


confussion matrix
[[22  5]
 [ 3 31]]


Accuracy of DNN: 86.88524590163934 

              precision    recall  f1-score   support

           0       0.88      0.81      0.85        27
           1       0.86      0.91      0.89        34

    accuracy                           0.87        61
   macro avg       0.87      0.86      0.87        61
weighted avg       0.87      0.87      0.87        61

